In [ ]:
# todo: schleife über alle drei bzw. vier text analysis klassen
# immer nur eine klasse at a time, weil sonst die GPU zu voll wird,
# dann immer ein artikel at a time (10 sind so das maximum, das auf einmal auf die GPU passt)
# vectorizer bracuth am meisten, summarizer eh
# dann normal über alle artikel laufen und hochladen, frage: gleich jeden artikel einzeln hochladen oder immer in chuncks
# da man immer neu in keykloak einloggen muss, würde ich sagen, immer so 100 verarbeiten (einzeln) und dann im 100er pack hochladen

In [1]:
from database_handling.DataDownload import DataDownloader
from database_handling.DataHandleAndOtherHelpers import DataHandler
from database_handling.DataUpload import DataUploader
from database_handling.KeycloakLogin import KeycloakLogin
from scrapers.SueddeutscheScraper import SueddeutscheScraper
from text_analysis.NEExtractor import NEExtractor
from text_analysis.Summarizer import Summarizer
from text_analysis.TopicExtractor import TopicExtractor
from text_analysis.Vectorizers import Vectorizer

In [2]:
import transformers
import json
import torch
import gc

In [3]:
#sueddeutsche_scraper = SueddeutscheScraper()
#sueddeutsche_scraper.start_browser()

In [4]:
#sueddeutsche_scraper.login()

In [5]:
#screenshot_path = "screenshot.png"
#sueddeutsche_scraper.driver.get_screenshot_as_file(screenshot_path)

In [6]:
#this_runs_articles = sueddeutsche_scraper.scrape_sueddeutsche()

In [7]:
import json

# Specify the path to your JSON dump file
json_file_path = 'sueddeutsche_articles.json'

# Open the file and load its content
with open(json_file_path, 'r') as file:
    this_runs_articles = json.load(file)

In [8]:
# get the token for the database
keycloak_login = KeycloakLogin()
token = keycloak_login.return_token()

In [9]:
# get all sueddeutsche urls that are already in the database
data_downloader = DataDownloader(token)
sueddeutsche_articles_in_db = data_downloader.get_content(url="https://www.sueddeutsche.de/") # TODO: Durch config ersetzen

In [10]:
#entity_extractor = NEExtractor()

In [11]:
#topic_extractor = TopicExtractor()

In [12]:
#summarizer = Summarizer()
vectorizer = Vectorizer()

No sentence-transformers model found with name deepset/gbert-base. Creating a new one with MEAN pooling.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
No sentence-transformers model found with name T-Systems-onsite/german-roberta-sentence-transformer-v2. Creating a new one with MEAN pooling.
No sentence-transformers model found with name xlm-roberta-large. Creating a new one with MEAN pooling.
No sentence-transformers model found with name google/bigbird-roberta-large. Creating a new one with MEAN pooling.
No sentence-transformers model found with name severinsimmler/xlm-roberta-longformer-large-16384. Creating a new one with MEAN pooling.


In [13]:
# instantiate the data handler
data_handler = DataHandler()

In [14]:
# for the articles that are already in the database, only update the last_verification_date
articles_for_last_verifcation_date_update = data_handler.find_scraped_articles_already_in_db(this_runs_articles, sueddeutsche_articles_in_db)
# safe the responses to the last verification date update
responses_to_last_verifcation_date_update = data_handler.patch_last_online_verification_date(token, articles_for_last_verifcation_date_update)
# get the articles that are not yet in the database
sueddeutsche_articles_not_yet_in_db = data_handler.find_scraped_articles_not_already_in_db(this_runs_articles, sueddeutsche_articles_in_db)
sueddeutsche_articles_not_yet_in_db_list_of_dicts = [article for article in this_runs_articles if article['url'] in sueddeutsche_articles_not_yet_in_db]

# Define the number of articles to process and upload per iteration
articles_per_iteration = 10

# Calculate the number of iterations
iterations = len(sueddeutsche_articles_not_yet_in_db_list_of_dicts) // articles_per_iteration + (len(sueddeutsche_articles_not_yet_in_db_list_of_dicts) % articles_per_iteration > 0)

In [15]:
articles = sueddeutsche_articles_not_yet_in_db_list_of_dicts[0:11]

In [ ]:
def delete_and_free_memory(obj):
    """
    Deletes the given object and frees GPU memory.
    
    Parameters:
    obj (object): The object to delete.
    """
    del obj  # Delete the object
    torch.cuda.empty_cache()  # Clear the GPU cache
    gc.collect()  # Force Python to clean up unused memory

In [ ]:
# Iterate over each article individually
for i, article in enumerate(sueddeutsche_articles_not_yet_in_db_list_of_dicts):
    print(f"Processing article {i + 1} of {len(sueddeutsche_articles_not_yet_in_db_list_of_dicts)}")

In [16]:
with torch.no_grad():
    #articles = entity_extractor.extract_entities(articles)
    #articles = topic_extractor.extract_topics(articles)
    articles = vectorizer.vectorize(articles)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Attention type 'block_sparse' is not possible if sequence_length: 73 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...
Input ids are automatically padded from 44 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 21 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 116 to 512 to be a multiple of `config.attentio

In [17]:
vectorizer = None
del vectorizer  # Delete the list holding articles
torch.cuda.empty_cache()  # Clears the GPU cache (if using PyTorch)
gc.collect()  # Force Python to clean up unused memory

0

In [10]:
jupyter_internal_vars = ['_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___']
for var in list(locals().keys()):
    if var not in jupyter_internal_vars and var not in ['torch', 'gc']:
        del locals()[var]

In [10]:
locals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__session__', '_i', '_ii', '_iii', '_i1', 'DataDownloader', 'DataHandler', 'DataUploader', 'KeycloakLogin', 'SueddeutscheScraper', 'NEExtractor', 'Summarizer', 'TopicExtractor', 'Vectorizer', '_i2', 'transformers', 'json', 'torch', 'gc', '_i3', 'json_file_path', 'file', 'this_runs_articles', '_i4', 'keycloak_login', 'token', '_i5', 'data_downloader', 'sueddeutsche_articles_in_db', '_i6', 'entity_extractor', 'topic_extractor', 'vectorizer', '_i7', 'data_handler', '_i8', 'articles_for_last_verifcation_date_update', 'responses_to_last_verifcation_date_update', 'sueddeutsche_articles_not_yet_in_db', 'sueddeutsche_articles_not_yet_in_db_list_of_dicts', 'articles_per_iteration', 'iterations', '_i9', 'articles', '_i10'])

In [ ]:
for i in range(iterations):
    print("Processing articles", i*articles_per_iteration, "to", (i+1)*articles_per_iteration, "from", len(sueddeutsche_articles_not_yet_in_db_list_of_dicts))
    # Get the articles for this iteration
    articles = sueddeutsche_articles_not_yet_in_db_list_of_dicts[i*articles_per_iteration:(i+1)*articles_per_iteration]

    print("Running text processing on articles", i*articles_per_iteration, "to", (i+1)*articles_per_iteration, "from", len(sueddeutsche_articles_not_yet_in_db_list_of_dicts))
    # Add the summaries, named entities, topics, and vectors to the articles dict
    #articles = summarizer.summarize(articles)
    with torch.no_grad():
        articles = entity_extractor.extract_entities(articles)
        articles = topic_extractor.extract_topics(articles)
        articles = vectorizer.vectorize(articles)

    # Remove main_text and lead_text from articles
    for article in articles:
        article.pop('main_text', None)
        article.pop('lead_text', None)


    print("Uploading articles", i*articles_per_iteration, "to", (i+1)*articles_per_iteration, "from", len(sueddeutsche_articles_not_yet_in_db_list_of_dicts))
    # Ensure that the token is still valid every n iterations
    # TODO: Tell Mario chuncking was done because I get a new token every 30 uploads to make sure the token is always valid
    # if we do that every 1 upload that takes much longer since it takes ~20 seconds to get a net token/ensure the token is valid
    keycloak_login = KeycloakLogin()
    token = keycloak_login.return_token()
    
    # Loop over articles and put every article into the database
    data_uploader = DataUploader(token)

    responses = []
    for article in articles:
        response = data_uploader.post_content(article)
        responses.append(response)

    print("Processed and uploaded articles", i*articles_per_iteration, "to", (i+1)*articles_per_iteration, "from", len(sueddeutsche_articles_not_yet_in_db_list_of_dicts))

    # Free up VRAM
    print_vram_usage()
    del articles  # Delete the list holding articles
    torch.cuda.empty_cache()  # Clears the GPU cache (if using PyTorch)
    gc.collect()  # Force Python to clean up unused memory
    print_vram_usage()
    
    # save the responses to a json file
    with open('responses.json', 'w') as f:
        json.dump(responses, f)

    # Free up VRAM
    print_vram_usage()
    del responses  # Delete the list holding articles
    torch.cuda.empty_cache()  # Clears the GPU cache (if using PyTorch)
    gc.collect()  # Force Python to clean up unused memory
    print_vram_usage()

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    model(input)
print(prof.key_averages().table(sort_by="cuda_time_total"))

In [ ]:
print_vram_usage()
del articles  # Delete the list holding articles
torch.cuda.empty_cache()  # Clears the GPU cache (if using PyTorch)
gc.collect()  # Force Python to clean up unused memory
print_vram_usage()

In [ ]:
response

In [ ]:
# write articles to json file
#import json
#with open('sueddeutsche_articles.json', 'w') as f:
#    json.dump(articles, f)